In [ ]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
from NetsVecino import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

In [ ]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

In [ ]:
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

In [ ]:
Cargar_errores = True



if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/error_mask y locs_buffer_act_vc-707/error_mask_054')
    

In [ ]:


NetVecinos = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=True, word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)

#Cuantizacion de los pesos

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NetVecinos.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
NetVecinos.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = NetVecinos, frac_bits = 11, int_bits = 4)
#loss,acc =NetVecinos.evaluate(test_dataset)

In [ ]:
write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]

diff_inp_output=[]
razon=[]
capa=[]
numero=[]
with pd.ExcelWriter('Analizando_fichero_detalle/diferencias_entre_resultados/Alexnet_Vecino.xlsx') as writer:
    
    for i,j in enumerate(write_layer):
        
        
        print('Capa',j,NetVecinos.layers[j].__class__.__name__)
        capa.append(NetVecinos.layers[j].__class__.__name__)
        numero.append(j)
        X = [x for x,y in test_dataset]
        #salidas del modelo sin fallas para la primer imagen del dataset de prueba
        out= get_all_outputs(NetVecinos,X[0])
        #print ('out',out[1])
        #out=get_all_outputs(Net2,iterator)
        #salidas del modelo con fallas para la primer imagen del dataset de prueba
        out_quantizacion = out[j-1]
        out_error_and_correction = out[j]
        out_input= tf.math.reduce_sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction )))
        diff_inp_output.append(out_input.numpy())
        
        out_output=tf.math.divide(tf.math.reduce_sum(tf.math.abs(out_quantizacion)),tf.math.reduce_sum(tf.math.abs(out_error_and_correction)))
        razon.append(out_output.numpy())
        
        print(' diferencia absoluta input y outputs: ', diff_inp_output)
        #print('razón absoluta entre input y outputs: ', tf.math.reduce_sum(tf.math.abs(tf.math.divide (out_quantizacion,out_error_and_correction ))))
        print('razón',razon)
    df_numero=pd.DataFrame(numero)
    df_capa = pd.DataFrame(capa)   
    df_inp_output=pd.DataFrame(diff_inp_output)
    df_razon=pd.DataFrame(razon)
        
        
    buf_diff_inp_out = pd.concat([df_numero,df_capa,df_inp_output,df_razon], axis=1, join='outer')
    buf_diff_inp_out.columns = ['numero','Capa','df_inp_output','df_razon']
    buf_diff_inp_out.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)
      